In [1]:
#import os
#os._exit(00)

In [2]:
#!pip uninstall accelerate transformers # https://stackoverflow.com/questions/76781329/importerror-with-transformers-and-accelerate-in-google-colab-seq2seqtrainingarg

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
!ls /content/drive/MyDrive/AI/models

In [ ]:
!pip install accelerate>=0.20.3 transformers
# then restart runtime if it does not work later on in the AutoModelForCausalLM
import os
os._exit(00)

In [ ]:
!pip install bitsandbytes
#!pip install accelerate
!pip install transformers
!pip install langchain
!pip install sentence-transformers
!pip install pypdf
!pip install chromadb
!pip install accelerate

In [6]:
#!pip install accelerate
#!pip install huggingface_hub
#pip install -i https://pypi.org/simple/ bitsandbytes

In [1]:
import torch
from transformers import BitsAndBytesConfig, AutoModelForCausalLM, AutoTokenizer, pipeline
from langchain import HuggingFacePipeline
from langchain import PromptTemplate, LLMChain

# Mistral LLM


In [8]:
!ls /content/drive/MyDrive/AI/models/mistral7B_4bit

config.json		mistral7B_4bit.pt  special_tokens_map.json  tokenizer.json
generation_config.json	model.safetensors  tokenizer_config.json    tokenizer.model


In [2]:
# if already downloaded:
path = "/content/drive/MyDrive/AI/models/mistral7B_4bit/"
#torch.save(model_4bit.state_dict(), path)
model_4bit = AutoModelForCausalLM.from_pretrained(path, device_map="auto")  # does its own .to(device)
tokenizer = AutoTokenizer.from_pretrained(path)


ImportError: Using `bitsandbytes` 8-bit quantization requires Accelerate: `pip install accelerate` and the latest version of bitsandbytes: `pip install -i https://pypi.org/simple/ bitsandbytes`

In [ ]:
print(f"Memory footprint: {model_4bit.get_memory_footprint() / 1e6:.2f} MB")

In [ ]:
# For the model quantization to be able to run in a free Google Colab notebook
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
)
# Version of the Mistral model used
model_id = "mistralai/Mistral-7B-Instruct-v0.1"

# Model and tokenizer
#model_4bit = AutoModelForCausalLM.from_pretrained( model_id, device_map="auto",quantization_config=quantization_config)
#tokenizer = AutoTokenizer.from_pretrained(model_id)

# So input sequence in model, embedded in integers, to tokens. tokenizer splits up sentence
# Model is no more than dictionary of tokens to integers. then inverse, integers get redone into tokens



In [ ]:
# Save the model & tokenizer
#path = F"/content/drive/MyDrive/AI/models/mistral7B_4bit"
#model_4bit.save_pretrained(path)
#tokenizer.save_pretrained(path)

In [ ]:
# Way to ask questions to the model:
def askQuestion(prompt, max_new_tokens = 500, temperature = 0.8, top_p = 0.9, do_sample=True, repetition_penalty=1.2):
  """ Encode the input prompt with Mistral instruct format, encode the prompt, use the Quantized Mistral model, decode the reponse and remove the input prompt from this response

      Args:
        prompt (str): Input prompt
        max_new_tokens (int): Specifies the maximum number of new tokens (words or pieces of words) that the model is allowed to generate in response to the prompt.
        temperature (float): Controls the randomness of the model's responses. Lower temperatures (closer to 0) make the model's outputs more deterministic and repetitive,
                             while higher temperatures (closer to 1 or above) increase diversity and creativity in the responses.
        top_p (float): Controls the model's sampling process, where it selects the next token based on a cumulative probability distribution of the possible next tokens.
                       A common setting might be top_p = 0.9, which means the model will only consider the smallest set of tokens whose cumulative probability exceeds 90%
                       for generating the next token. This approach helps to focus the model's choices on more likely outcomes, reducing the
                       chance of selecting very improbable tokens and generally producing more coherent text.

        do_sample (bool): The do_sample parameter controls whether the Large Language Model (LLM) generates text based on a sampling approach rather than simply
                          choosing the most likely next token at each step. When do_sample is set to True, the model uses the probabilities of each token to randomly
                          generate the next token in the sequence. This method introduces variability and creativity in the generated text, making the outputs less
                          deterministic and more diverse.
        repetition_penalty (float): is used to discourage or encourage the repetition of tokens (words or pieces of words) in the text generated by the model.
                                   It modifies the likelihood of tokens based on their previous occurrences in the generated text,
                                   aiming to control redundancy and improve the diversity and quality of the output.

      Returns:
        (str): Clean response to the prompt
  """
  prompt = f"""[INST]{prompt}[/INST]"""
  input = tokenizer.encode(prompt, return_tensors="pt").to("cuda")  # should look online how tokenizer works
  output = model_4bit.generate(input, max_new_tokens=max_new_tokens, temperature=temperature, top_p=top_p, repetition_penalty=repetition_penalty, do_sample=do_sample)
  answer = tokenizer.decode(output[0],skip_special_tokens=True).replace(prompt, "" )
  return answer

  # temperature = stochastic influence of generating next word

### Asking Questions

In [ ]:
question = "Enumerate the events between the invasion of ukraine by russia since 2022"
response = askQuestion(question,temperature = 0, top_p = 1.0, do_sample=False, repetition_penalty=0.05) # no creativity, almost totally deterministic
print(response)

In [ ]:
# More creative. parameters are
question = "Enumerate the events between the invasion of ukraine by russia since 2022. Put the dates in front."
response = askQuestion(question)
print(response)

## Other model, Google Gemma (open source Gemini)
